In [ ]:
import os
from du_astro_utils import calibration, photometry, utils
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from scipy.ndimage import median_filter
from tqdm import tqdm

## Reduce Images

In [ ]:
target_type = "variable_star"
if target_type == "asteroid":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_ASTER)
elif target_type == "exoplanet":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_EXPLTS)
elif target_type == "variable_star":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_VARSTARS)
elif target_type == "galaxy_cluster":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_GALCLUST)
elif target_type == "cluster":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_CLUSTERS)
os.listdir(data_dir)

In [ ]:
for ix, ddir in enumerate(os.listdir(data_dir)):
    subdata_dir = os.path.join(data_dir, ddir)
    if os.path.isdir(subdata_dir):
        list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
        list_fits = sorted(list_fits)
        print(f"{ix} - {subdata_dir} : {len(list_fits)} files")

In [ ]:
achoice = os.listdir(data_dir)[7]
subdata_dir = os.path.join(data_dir, achoice)
list_fits = sorted([im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im])
list_fits
im0 = fits_sci_image = os.path.join(subdata_dir, list_fits[0])
with fits.open(im0) as hdul:
    hdr = hdul[0].header
# print(hdr.keys)
hdr.get("FOCPOS")

In [ ]:
for idx in range(7, 11):
    achoice = os.listdir(data_dir)[idx]
    reduced = False
    aligned = False
    subdata_dir = os.path.join(data_dir, achoice)
    if reduced:
        subdata_dir = os.path.join(subdata_dir, "REDUCED")
    if aligned:
        subdata_dir = os.path.join(subdata_dir, "aligned")
    list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
    list_fits = sorted(list_fits)

    rerun = False
    test_mode = False  # Only runs on a few images, prints the tables at each step and does not write files.
    # dict_of_dict = {}

    if test_mode:
        list_fits = list_fits[:10]
    for loc, scimage in enumerate(tqdm(list_fits)):
        fits_sci_image = os.path.join(subdata_dir, scimage)
        fits_sci_image = os.path.abspath(fits_sci_image)
        if os.path.isfile(fits_sci_image):
            # Get image directory, failename and extension
            sc_im_dir = os.path.abspath(os.path.dirname(fits_sci_image))
            sc_im_name, sc_im_ext = os.path.splitext(os.path.basename(fits_sci_image))

            bias_dir, darks_dir, flats_dir = utils.get_calib_dirs_photometry(fits_sci_image)
            # print(bias_dir, darks_dir, flats_dir)

            # Get information from FITS header
            # sc_date, sc_scope, sc_cam, sc_filter, sc_focus, sc_expos, sc_x, sc_y = calibration.get_infos_from_image(fits_sci_image, verbose=True)
            # print(sc_date, sc_filter, sc_focus, sc_expos)
            # flats_list = calibration.load_flat_frames(flats_dir, sc_date, sc_cam, sc_focus, sc_filter, sc_x, sc_y, verbose=True)
            # for flat in flats_list:
            #    with fits.open(flat) as hdul:
            #        hdr = hdul[0].header
            #        print(hdr.get('FOCPOS'), hdr.get('INSTFILT'))

            # Run calibration
            # dico_calib = calibration.dedark_sci_image(fits_sci_image, override_date_check=True, max_days=7,\
            #                                          overwrite=rerun, verbose=False, write_tmp=test_mode,\
            #                                          overwrite_calibs=False)
            calibration.reduce_sci_image(fits_sci_image, darks_dir, flats_dir, bias_dir, override_date_check=False, max_days=10, overwrite=rerun, verbose=False, write_tmp=test_mode, overwrite_calibs=(rerun and loc == 0))
            # dict_of_dict.update({sc_im_name: dico_calib})

__If not done, plate-solve and align within AIJ.__

## Plate solve missing WCS

In [ ]:
data_dir = os.path.join(utils.C2PU_RES_DIR, utils.DIR_PHOTOM, utils.DIR_VARSTARS)

In [ ]:
reduced = True
aligned = False
for ix, ddir in enumerate(os.listdir(data_dir)):
    subdata_dir = os.path.join(data_dir, ddir)
    if reduced:
        subdata_dir = os.path.join(subdata_dir, "REDUCED")
    if aligned:
        subdata_dir = os.path.join(subdata_dir, "aligned")
    if os.path.isdir(subdata_dir):
        list_fits = [im for im in sorted(os.listdir(subdata_dir)) if "REDUCED.fits" in im]
        list_fits = sorted(list_fits)
        print(f"{ix} - {subdata_dir} : {len(list_fits)} files")
    else:
        print(f"{subdata_dir} : not a directory.")

In [ ]:
dirchoice = os.listdir(data_dir)[2]
subdata_dir = os.path.join(data_dir, dirchoice)
if reduced:
    subdata_dir = os.path.join(subdata_dir, "REDUCED")
if aligned:
    subdata_dir = os.path.join(subdata_dir, "aligned")
if os.path.isdir(subdata_dir):
    list_fits = [im for im in sorted(os.listdir(subdata_dir)) if "REDUCED.fits" in im]
    list_fits = sorted(list_fits)

In [ ]:
list_fits

In [ ]:
rel_data_dir = os.path.relpath(subdata_dir)
for image in list_fits:
    image_to_solve = os.path.join(rel_data_dir, image)
    solved_image = "{0}_solved{1}".format(*os.path.splitext(image_to_solve))
    solve_cmd = f"solve-field --overwrite --downsample 2 --no-plot --tweak-order 3 --scale-low 0.3 --scale-high 2 --cpulimit 10 --no-remove-lines --uniformize 0 --config /etc/astrometry.cfg -N {solved_image} {image_to_solve}"
    os.system(f"{solve_cmd} > log_solve_field_{image}.log")

## Stacking images

In [ ]:
dirchoice = os.listdir(data_dir)[2]
subdata_dir = os.path.join(data_dir, dirchoice)
if reduced:
    subdata_dir = os.path.join(subdata_dir, "REDUCED")
if aligned:
    subdata_dir = os.path.join(subdata_dir, "aligned")
if os.path.isdir(subdata_dir):
    list_fits = [im for im in sorted(os.listdir(subdata_dir)) if "REDUCED" in im and ".fits" in im]
    list_fits = sorted(list_fits)
list_fits

In [ ]:
for filt in ["g"]:
    exp = 300
    rel_data_dir = os.path.relpath(subdata_dir)
    if not os.path.isfile(f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED.fits"):
        swarp_cmd = f"swarp {rel_data_dir}/*SDSS{filt}*{exp:04d}s*REDUCED*.fits -c default.swarp -IMAGEOUT_NAME coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED.fits -CENTER_TYPE ALL"
        os.system(swarp_cmd)

In [ ]:
if True:
    for filt in ["g", "r", "i", "z"]:
        exp = 30 if filt == "i" else 15  # 5
        with fits.open(f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED.fits") as stack_ip:
            hdr = stack_ip[0].header
            data = stack_ip[0].data
        mean, med, sigma = sigma_clipped_stats(data, sigma=3)
        plt.imshow(data, cmap="gray", vmin=med - 5 * sigma, vmax=med + 5 * sigma)
        plt.colorbar()
        plt.show()

## References in PANSTARRS

In [ ]:
from du_astro_utils import query_panstarrs

In [ ]:
ref_ps = query_panstarrs(f"coadd_{dirchoice}_SDSSgp_RED.fits")

In [ ]:
ref_ps

In [ ]:
from astropy.wcs import WCS
from astropy.wcs.utils import skycoord_to_pixel
from astropy.coordinates import Angle, SkyCoord

coord_panstarrs = SkyCoord(ref_ps["RAJ2000"], ref_ps["DEJ2000"])

## Photometry

### G-band

In [ ]:
from astropy.table import Table, vstack

from astropy.time import Time
import warnings
from astropy.utils.exceptions import AstropyWarning
from astropy.coordinates.name_resolve import NameResolveError

use_sextractor = True

red_sci_image = f"coadd_{dirchoice}_SDSSgp_RED.fits"
# red_sci_image = os.path.relpath(os.path.join(subdata_dir, 'aligned_NGC-7317_20211007T221309617_SC_SDSSgp+_0045s000_000000_REDUCED.fits'))
im_dir = os.path.abspath(os.path.dirname(red_sci_image))
im_name, im_ext = os.path.splitext(os.path.basename(red_sci_image))
with fits.open(red_sci_image) as hdul:
    hdu = hdul[0]
    wcs = WCS(hdu.header)
    epoch = Time(hdu.header.get("MJD-OBS"), format="mjd", scale="utc")
    if use_sextractor:
        sex_cmd = f"source-extractor -c default.sex {red_sci_image} -CATALOG_NAME coadd_{dirchoice}_SDSSgp_RED.cat -CATALOG_TYPE FITS_1.0 -VERBOSE_TYPE QUIET"
        os.system(sex_cmd)
        cat_tab_g = Table.read(f"coadd_{dirchoice}_SDSSgp_RED.cat")
        cat_tab_g.rename_column("X_IMAGE", "xcentroid")
        cat_tab_g.rename_column("Y_IMAGE", "ycentroid")
        _, fwhm, _ = sigma_clipped_stats(cat_tab_g["FWHM_IMAGE"], sigma=3)
        source_coords_g = SkyCoord(ra=cat_tab_g["ALPHA_J2000"], dec=cat_tab_g["DELTA_J2000"], unit="deg", obstime=epoch)
    else:
        sources_g = photometry.detect_sources(red_sci_image, detection_fwhm=10, verbose=False)
        try:
            fwhm = photometry.get_fwhm(red_sci_image, sources)
        except RuntimeError:
            fwhm = 10
        cat_tab_g = photometry.apert_photometry(red_sci_image, sources_g, fwhm)
        source_coords_g = SkyCoord.from_pixel(cat_tab_g["xcenter"], cat_tab_g["xcenter"], wcs)

In [ ]:
cat_tab_g

In [ ]:
plt.scatter(cat_tab_g["CLASS_STAR"], cat_tab_g["ELONGATION"])
plt.ylim(0.0, 5.0)

In [ ]:
plt.hist(cat_tab_g["CLASS_STAR"], bins=50)

In [ ]:
cat_stars_g = cat_tab_g[cat_tab_g["CLASS_STAR"] > 0.4]
star_coords_g = source_coords_g[cat_tab_g["CLASS_STAR"] > 0.4]

In [ ]:
cat_stars_g

In [ ]:
import astropy.units as u

xm_id, xm_ang_distance, _ = star_coords_g.match_to_catalog_sky(coord_panstarrs, nthneighbor=1)
# print(hdu.header.get('PIXSCALX') * fwhm)
# max_sep = hdu.header.get('PIXSCALX') * fwhm * u.arcsec
max_sep = 2.5 * u.arcsec
sep_constraint = xm_ang_distance < max_sep
coord_matches_g = star_coords_g[sep_constraint]
catalog_matches_g = ref_ps[xm_id[sep_constraint]]
coord_catalog_matches_g = coord_panstarrs[xm_id[sep_constraint]]

In [ ]:
# Compute instrumental magnitude
if not use_sextractor:
    exptime = hdu.header.get("EXPTIME")
    ins_mag_g = -2.5 * np.log10(cat_stars_g[sep_constraint]["aper_sum_bkgsub"] / exptime)
    cat_mag_g = ref_ps["gmag"][xm_id[sep_constraint]]

    ins_err_g = ins_mag_g - -2.5 * np.log10((cat_stars_g[sep_constraint]["aper_sum_bkgsub"] + cat_stars_g[sep_constraint]["noise"]) / exptime)
    cat_err_g = ref_ps["e_gmag"][xm_id[sep_constraint]]

    cat_stars_g["ins_mag"] = 99
    cat_stars_g["ins_mag"][sep_constraint] = ins_mag_g
else:
    cat_mag_g = ref_ps["gmag"][xm_id[sep_constraint]]
    cat_err_g = ref_ps["e_gmag"][xm_id[sep_constraint]]
    ins_mag_g = cat_stars_g[sep_constraint]["MAG_AUTO"]
    ins_err_g = cat_stars_g[sep_constraint]["MAGERR_AUTO"]

In [ ]:
sel = ins_mag_g < 99
plt.scatter(ins_mag_g[sel], cat_mag_g[sel])

In [ ]:
plt.scatter(cat_mag_g[sel], ins_mag_g[sel] - cat_mag_g[sel])

In [ ]:
from sklearn import linear_model

# Selection from magnitude range
mag_min, mag_max = 12, 19
cond = (cat_mag_g > mag_min) & (cat_mag_g < mag_max) & (~cat_mag_g.mask) & (~np.isnan(ins_mag_g)) & (ins_mag_g < 99)

# Create two mock arrays for linear regression
X = ins_mag_g[cond].reshape(-1, 1)
y = cat_mag_g[cond].reshape(-1, 1)


# Simple linear regression
linear = linear_model.LinearRegression()
linear.fit(X, y)


# sigma clipping pour choisir le threshold
from scipy import stats

MAD = stats.median_abs_deviation(X - y)
_, _, sig = sigma_clipped_stats(X - y)

print(MAD, sig)

# RANSAC linear regressions
ransac = linear_model.RANSACRegressor(residual_threshold=3 * MAD[0])
# ransac = linear_model.RANSACRegressor()
ransac.fit(X, y)

# Results
print("Photometric calibration:")
print(f"  Linear Slope: {linear.coef_[0][0]:.3f}")
print(f"  Linear ZP   : {linear.intercept_[0]:.3f}\n")
print(f"  RANSAC Slope: {ransac.estimator_.coef_[0][0]:.3f}")
print(f"  RANSAC ZP   : {ransac.estimator_.intercept_[0]:.3f}")

In [ ]:
# Plotting regression
# Outliers and Valid points
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

# Linear regressions (simple and RANSAC)
line_X = np.arange(X.min(), X.max() + 1)[:, np.newaxis]
line_y_simple = linear.predict(line_X)
line_y_ransac = ransac.predict(line_X)

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Plot data
ax[0].scatter(X[inlier_mask], y[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[0].scatter(X[outlier_mask], y[outlier_mask], color="gray", marker=".", label="Outliers")

# Plot regressions
ax[0].plot(line_X, line_y_simple, color="cornflowerblue", label="Linear regressor")
ax[0].plot(line_X, line_y_ransac, color="navy", label="RANSAC regressor")

# Axes...
ax[0].legend(loc="lower right")
# ax[0].set_ylim([10,18])
ax[0].set_xlabel("Instrument magnitude")
ax[0].set_ylabel("Catalog magnitude")
ax[0].set_aspect("equal")

_, zp_median, zp_sigma = sigma_clipped_stats(y - X, sigma=3)
ax[1].scatter(y[inlier_mask], y[inlier_mask] - X[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[1].scatter(y[outlier_mask], y[outlier_mask] - X[outlier_mask], color="gray", marker=".", label="Outliers")
ax[1].set_xlabel("Catalog magnitude")

ax[1].axhline(zp_median, label="Median")
ax[1].axhline(zp_median + zp_sigma, linestyle="--", label="Standard deviation")
ax[1].axhline(zp_median - zp_sigma, linestyle="--")
print(f"  sigma  ZP   : {zp_sigma:.3f}")

ax[1].set_ylabel("Instrument - Catalog magnitude")
ax[1].legend(loc="best")

In [ ]:
# Compute calibrated mag
cat_tab_g["AB_MAG"] = 99.0

# Positive values
if not use_sextractor:
    positive = np.where(cat_tab_g["aper_sum_bkgsub"] > 0)
    cat_tab_g["AB_MAG"][positive] = ransac.predict((-2.5 * np.log10(cat_tab_g[positive]["aper_sum_bkgsub"] / exptime)).data.reshape(-1, 1)).flatten()
else:
    positive = np.where(cat_tab_g["FLUX_AUTO"] > 0)
    cat_tab_g["AB_MAG"][positive] = ransac.predict(cat_tab_g[positive]["MAG_AUTO"].data.reshape(-1, 1)).flatten()
cat_tab_g

### R- band

In [ ]:
red_sci_image = f"coadd_{dirchoice}_SDSSrp_RED.fits"
# red_sci_image = os.path.relpath(os.path.join(subdata_dir, 'aligned_NGC-7317_20211007T215225971_SC_SDSSrp+_0060s000_000000_REDUCED.fits'))
im_dir = os.path.abspath(os.path.dirname(red_sci_image))
im_name, im_ext = os.path.splitext(os.path.basename(red_sci_image))
with fits.open(red_sci_image) as hdul:
    hdu = hdul[0]
    wcs = WCS(hdu.header)
    epoch = Time(hdu.header.get("MJD-OBS"), format="mjd")
    if use_sextractor:
        sex_cmd = f"source-extractor -c default.sex {red_sci_image} -CATALOG_NAME coadd_{dirchoice}_SDSSrp_RED.cat -CATALOG_TYPE FITS_1.0 -VERBOSE_TYPE QUIET"
        os.system(sex_cmd)
        cat_tab_r = Table.read(f"coadd_{dirchoice}_SDSSrp_RED.cat")
        cat_tab_r.rename_column("X_IMAGE", "xcentroid")
        cat_tab_r.rename_column("Y_IMAGE", "ycentroid")
        _, fwhm, _ = sigma_clipped_stats(cat_tab_r["FWHM_IMAGE"], sigma=3)
        source_coords_r = SkyCoord(ra=cat_tab_r["ALPHA_J2000"], dec=cat_tab_r["DELTA_J2000"], unit="deg", obstime=epoch)
    else:
        sources_r = photometry.detect_sources(red_sci_image, detection_fwhm=10, verbose=False)
        try:
            fwhm = photometry.get_fwhm(red_sci_image, sources_r)
        except RuntimeError:
            fwhm = 10
        cat_tab_r = photometry.apert_photometry(red_sci_image, sources_r, fwhm)
        source_coords_r = SkyCoord.from_pixel(cat_tab_r["xcenter"], cat_tab_r["xcenter"], wcs)

In [ ]:
plt.scatter(cat_tab_r["CLASS_STAR"], cat_tab_r["ELONGATION"])
plt.ylim(0.0, 5.0)

In [ ]:
plt.hist(cat_tab_r["CLASS_STAR"], bins=50)

In [ ]:
cat_stars_r = cat_tab_r[cat_tab_r["CLASS_STAR"] > 0.1]
star_coords_r = source_coords_r[cat_tab_r["CLASS_STAR"] > 0.1]

In [ ]:
xm_id, xm_ang_distance, _ = star_coords_r.match_to_catalog_sky(coord_panstarrs, nthneighbor=1)
# print(hdu.header.get('PIXSCALX') * fwhm)
# max_sep = hdu.header.get('PIXSCALX') * fwhm * u.arcsec
max_sep = 2.5 * u.arcsec
sep_constraint = xm_ang_distance < max_sep
coord_matches_r = star_coords_r[sep_constraint]
catalog_matches_r = ref_ps[xm_id[sep_constraint]]
coord_catalog_matches_r = coord_panstarrs[xm_id[sep_constraint]]

In [ ]:
# Compute instrumental magnitude
if not use_sextractor:
    exptime = hdu.header.get("EXPTIME")
    ins_mag_r = -2.5 * np.log10(cat_stars_r[sep_constraint]["aper_sum_bkgsub"] / exptime)
    cat_mag_r = ref_ps["rmag"][xm_id[sep_constraint]]

    ins_err_r = ins_mag_r - -2.5 * np.log10((cat_stars_r[sep_constraint]["aper_sum_bkgsub"] + cat_stars_r[sep_constraint]["noise"]) / exptime)
    cat_err_r = ref_ps["e_rmag"][xm_id[sep_constraint]]

    cat_stars_r["ins_mag"] = 99
    cat_stars_r["ins_mag"][sep_constraint] = ins_mag_r
else:
    cat_mag_r = ref_ps["rmag"][xm_id[sep_constraint]]
    cat_err_r = ref_ps["e_rmag"][xm_id[sep_constraint]]
    ins_mag_r = cat_stars_r[sep_constraint]["MAG_AUTO"]
    ins_err_r = cat_stars_r[sep_constraint]["MAGERR_AUTO"]

sel = ins_mag_r < 99
plt.scatter(ins_mag_r[sel], cat_mag_r[sel])

In [ ]:
# Selection from magnitude range
mag_min, mag_max = 8, 19
cond = (cat_mag_r > mag_min) & (cat_mag_r < mag_max) & (~cat_mag_r.mask) & (~np.isnan(ins_mag_r)) & (ins_mag_r < 99)

# Create two mock arrays for linear regression
X = ins_mag_r[cond].reshape(-1, 1)
y = cat_mag_r[cond].reshape(-1, 1)


# Simple linear regression
linear = linear_model.LinearRegression()
linear.fit(X, y)


# sigma clipping pour choisir le threshold
from scipy import stats

MAD = stats.median_abs_deviation(X - y)
_, _, sig = sigma_clipped_stats(X - y)

print(MAD, sig)

# RANSAC linear regressions
ransac = linear_model.RANSACRegressor(residual_threshold=3 * MAD[0])
# ransac = linear_model.RANSACRegressor()
ransac.fit(X, y)

# Results
print("Photometric calibration:")
print(f"  Linear Slope: {linear.coef_[0][0]:.3f}")
print(f"  Linear ZP   : {linear.intercept_[0]:.3f}\n")
print(f"  RANSAC Slope: {ransac.estimator_.coef_[0][0]:.3f}")
print(f"  RANSAC ZP   : {ransac.estimator_.intercept_[0]:.3f}")

# Plotting regression
# Outliers and Valid points
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

# Linear regressions (simple and RANSAC)
line_X = np.arange(X.min(), X.max() + 1)[:, np.newaxis]
line_y_simple = linear.predict(line_X)
line_y_ransac = ransac.predict(line_X)

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Plot data
ax[0].scatter(X[inlier_mask], y[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[0].scatter(X[outlier_mask], y[outlier_mask], color="gray", marker=".", label="Outliers")

# Plot regressions
ax[0].plot(line_X, line_y_simple, color="cornflowerblue", label="Linear regressor")
ax[0].plot(line_X, line_y_ransac, color="navy", label="RANSAC regressor")

# Axes...
ax[0].legend(loc="lower right")
# ax[0].set_ylim([10,18])
ax[0].set_xlabel("Instrument magnitude")
ax[0].set_ylabel("Catalog magnitude")
ax[0].set_aspect("equal")

_, zp_median, zp_sigma = sigma_clipped_stats(y - X, sigma=3)
ax[1].scatter(y[inlier_mask], y[inlier_mask] - X[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[1].scatter(y[outlier_mask], y[outlier_mask] - X[outlier_mask], color="gray", marker=".", label="Outliers")
ax[1].set_xlabel("Catalog magnitude")

ax[1].axhline(zp_median, label="Median")
ax[1].axhline(zp_median + zp_sigma, linestyle="--", label="Standard deviation")
ax[1].axhline(zp_median - zp_sigma, linestyle="--")
print(f"  sigma  ZP   : {zp_sigma:.3f}")

ax[1].set_ylabel("Instrument - Catalog magnitude")
ax[1].legend(loc="best")

In [ ]:
# Compute calibrated mag
cat_tab_r["AB_MAG"] = 99.0

# Positive values
if not use_sextractor:
    positive = np.where(cat_tab_r["aper_sum_bkgsub"] > 0)
    cat_tab_r["AB_MAG"][positive] = ransac.predict((-2.5 * np.log10(cat_tab_r[positive]["aper_sum_bkgsub"] / exptime)).data.reshape(-1, 1)).flatten()
else:
    positive = np.where(cat_tab_r["FLUX_AUTO"] > 0)
    cat_tab_r["AB_MAG"][positive] = ransac.predict(cat_tab_r[positive]["MAG_AUTO"].data.reshape(-1, 1)).flatten()
cat_tab_r

## Cross-match R et G

In [ ]:
xm_id, xm_ang_distance, _ = source_coords_r.match_to_catalog_sky(source_coords_g, nthneighbor=1)
# print(hdu.header.get('PIXSCALX') * fwhm)
max_sep = 2.5 * u.arcsec
sep_constraint = xm_ang_distance < max_sep
coord_matches = source_coords_r[sep_constraint]
catalog_matches = cat_tab_g[xm_id[sep_constraint]]
coord_catalog_matches = source_coords_g[xm_id[sep_constraint]]

In [ ]:
g_cat = catalog_matches
r_cat = cat_tab_r[sep_constraint]

sel = np.logical_and(g_cat["AB_MAG"] < 99.0, r_cat["AB_MAG"] < 99.0)
plt.scatter(g_cat[sel]["AB_MAG"] - r_cat[sel]["AB_MAG"], g_cat[sel]["AB_MAG"], marker=".")

On a ici un mélange d'étoiles et de galaxies. Il faut construire le catalogue de galaxies pour la suite.

In [ ]:
plt.scatter(g_cat["CLASS_STAR"], g_cat["ELONGATION"], marker=".")
plt.scatter(r_cat["CLASS_STAR"], r_cat["ELONGATION"], marker="x", alpha=0.5)
plt.ylim(0.0, 5.0)

In [ ]:
joined_cat = g_cat.copy()

In [ ]:
joined_cat.columns

In [ ]:
joined_cat.rename_column("FLUX_AUTO", "FLUX_AUTO_G")
joined_cat.rename_column("FLUXERR_AUTO", "FLUXERR_AUTO_G")
joined_cat.rename_column("MAG_AUTO", "MAG_AUTO_G")
joined_cat.rename_column("MAGERR_AUTO", "MAGERR_AUTO_G")
joined_cat.rename_column("AB_MAG", "AB_MAG_G")

In [ ]:
joined_cat["FLUX_AUTO_R"] = r_cat["FLUX_AUTO"]
joined_cat["FLUXERR_AUTO_R"] = r_cat["FLUXERR_AUTO"]
joined_cat["MAG_AUTO_R"] = r_cat["MAG_AUTO"]
joined_cat["MAGERR_AUTO_R"] = r_cat["MAGERR_AUTO"]

In [ ]:
joined_cat["AB_MAG_R"] = r_cat["AB_MAG"]

In [ ]:
joined_cat

In [ ]:
gal_cat = joined_cat[joined_cat["CLASS_STAR"] < 0.3]
sel = np.logical_and(gal_cat["AB_MAG_G"] < 99.0, gal_cat["AB_MAG_R"] < 99.0)
gal_cat = gal_cat[sel]

In [ ]:
gal_cat

In [ ]:
plt.scatter(gal_cat["AB_MAG_G"] - gal_cat["AB_MAG_R"], gal_cat["AB_MAG_G"])

In [ ]:
plt.hist(gal_cat["AB_MAG_G"] - gal_cat["AB_MAG_R"])

In [ ]:
gal_cat.write(f"galaxies_{dirchoice}_G_R_RED.fits", format="fits", overwrite=True)

In [ ]:
with fits.open(f"coadd_{dirchoice}_SDSSgp_RED.fits") as stack_gp:
    hdr = stack_gp[0].header
    data = stack_gp[0].data
mean, med, sigma = sigma_clipped_stats(data, sigma=3)
plt.imshow(data, cmap="gray", vmin=sigma, vmax=med + 5 * sigma)
plt.colorbar()
plt.scatter(gal_cat["xcentroid"], gal_cat["ycentroid"], color="y", alpha=0.3)